In [2]:
# Importing the required libraries for all the notebooks in this project

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import statsmodels as sm
import polars as pl
import scipy
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import ttest_ind
from matplotlib.pyplot import tick_params

pl.enable_string_cache()
pl.Config.set_streaming_chunk_size(10000)

polars.config.Config

In [3]:
pl.enable_string_cache()

# Set the streaming chunk size
pl.Config.set_streaming_chunk_size(10000)

columns = [
    "Epoch",
    "range_km",
    "Mag",
    "Track",
    "epsecs",
    "Satellite",
    "sat_j2000",
    "obs_j2000",
    "az_rad",
    "el_rad",
    'phase_angle_rad'
]

N = 300000000
lf = pl.scan_parquet("mmt.parquet").limit(n=N).select(columns)

df = lf.collect()
print(df.head())

shape: (5, 11)
┌─────────────┬────────────┬─────────┬─────────┬───┬────────────┬──────────┬──────────┬────────────┐
│ Epoch       ┆ range_km   ┆ Mag     ┆ Track   ┆ … ┆ obs_j2000  ┆ az_rad   ┆ el_rad   ┆ phase_angl │
│ ---         ┆ ---        ┆ ---     ┆ ---     ┆   ┆ ---        ┆ ---      ┆ ---      ┆ e_rad      │
│ datetime[ms ┆ f32        ┆ f32     ┆ u32     ┆   ┆ array[f32, ┆ f32      ┆ f32      ┆ ---        │
│ , UTC]      ┆            ┆         ┆         ┆   ┆ 3]         ┆          ┆          ┆ f32        │
╞═════════════╪════════════╪═════════╪═════════╪═══╪════════════╪══════════╪══════════╪════════════╡
│ 2014-08-17  ┆ 1193.54748 ┆ 4.91212 ┆ 1576304 ┆ … ┆ [4250.5957 ┆ 4.994911 ┆ 0.549699 ┆ 0.853081   │
│ 01:07:10.15 ┆ 5          ┆         ┆         ┆   ┆ 03, 1834.4 ┆          ┆          ┆            │
│ 3 UTC       ┆            ┆         ┆         ┆   ┆ 79736,     ┆          ┆          ┆            │
│             ┆            ┆         ┆         ┆   ┆ 437…       ┆          ┆

In [4]:
#Sample the DataFrame
sample_rate = 0.001
sampled_df = df.sample(fraction=sample_rate, seed=42)

In [5]:
desc_df = df.describe()
desc_sampled_df = sampled_df.describe()

## Comparing Sampled and Original DataFrame
This is to confirm that the sampled dataframe is a good representation of the original dataframe.
Even though the sample is random, by setting the seed we can produce a predictable sample each time.

In [20]:
numeric_types = [pl.Float64, pl.Int64]
numeric_columns = [column_name for column_name in desc_df.columns
                        if desc_df[column_name].dtype in numeric_types]

desc_df = desc_df.select(numeric_columns)
desc_sampled_df = desc_sampled_df.select(numeric_columns)

# Calculating the difference between the original and sampled DataFrames
diff_df = desc_df - desc_sampled_df

diff_df = diff_df.with_columns(pl.lit(desc_df['statistic']).alias('statistical_method'))

final_diff_df = diff_df.select(['statistical_method', 'range_km', 'Mag', 'epsecs', 'az_rad', 'el_rad', 'phase_angle_rad'])

final_diff_df

statistical_method,range_km,Mag,epsecs,az_rad,el_rad,phase_angle_rad
str,f64,f64,f64,f64,f64,f64
"""count""",2.78354865e8,2.78354865e8,2.78354865e8,2.78354865e8,2.78354865e8,2.78354865e8
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",-9.966797,-0.000007,0.794464,-0.002712,-0.00028,0.000491
"""std""",-22.34375,0.000887,-1.713379,-0.000327,-0.000046,-0.000078
"""min""",-66.51059,-0.65757,0.0,-0.000017,-0.000479,-0.008983
"""25%""",0.025024,-0.00297,-0.097,-0.00637,-0.000304,0.00029
"""50%""",3.798096,-0.00098,0.104004,-0.002287,-0.000602,0.001348
"""75%""",-24.079102,0.00218,2.088013,-0.002459,0.000017,-0.000223
"""max""",12.96875,0.981,698.699219,0.000008,0.000378,0.05932


## Observations
Since the random sample does not significantly alter the means or quantiles of the column values, we can replace the original dataframe with the sampled dataframe.

### Final Method to Load Data (replicated in every notebook)
Pre-sampling the dataframe and then selecting it using Lazyframe and then collect it.

In [ ]:
sample_rate = 0.001
lf = lf.with_row_index("row_num")

sampled_lf = lf.filter(pl.col("row_num") % int(1/sample_rate) == 0)
sampled_df = sampled_lf.collect()

print(sampled_df.head())